## GroupBy  
O nome "group by" vem de um comando na linguagem do banco de dados SQL, mas talvez seja mais esclarecedor pensar nisso nos termos cunhados primeiramente por Hadley Wickham, da Rstats: **split, apply, combine** (dividir, aplicar, combinar).

![](Split, apply, combine.png)

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns # Vamos importar o seaborn apenas para utilizar o dataset planet

In [2]:
planets = sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


**GroupBy**  
Podemos calcular a operação mais básica split-apply-combine com o método ``groupby()`` de DataFrames, passando o nome da coluna-chave desejada.  
A função ``groupby()`` retornará somente um objeto (DataFrameGroupBy). Para visualizarmos o resultado necessitamos informar o agregador para esse objeto, que pode ser ``sum()``, ``median()``, ``count()``, etc. 

In [3]:
# No exemplo a seguir agruparemos pela coluna method e calcularemos a mediana da coluna orbital_period
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [4]:
# No exemplo abaixo agrupamos por método e ano e contamos os valores das demais colunas.
# Mostramos apenas as primeiras 10 linhas para a visualização não ficar muito grande
planets2 = planets.groupby(['method', 'year']).count()
planets2.head(10)

number  orbital_period  mass  distance
method                    year                                        
Astrometry                2010       1               1     0         1
                          2013       1               1     0         1
Eclipse Timing Variations 2008       2               2     0         2
                          2009       1               1     1         0
                          2010       2               2     0         2
                          2011       3               3     0         0
                          2012       1               1     1         0
Imaging                   2004       3               0     0         3
                          2005       1               0     0         1
                          2006       4               1     0         2

**Aggregate, filter, transform, apply**  
Em particular, os objetos GroupBy possuem os métodos ``aggregate()``, ``filter()``, ``transform()`` e ``apply()`` que implementam eficientemente uma variedade de operações úteis antes de combinar os dados agrupados.

**Aggregate**  
Agora, estamos familiarizados com as agregações do GroupBy com ``sum()``, ``median()`` e semelhantes, mas o método ``aggregate()`` permite ainda mais flexibilidade. Podemos usar uma string, uma função ou uma lista e calcular todos os agregados de uma só vez.

In [36]:
planets.groupby('method').aggregate({'distance': ['min', 'mean', 'max'], 'orbital_period': [np.median]})

distance                       orbital_period
                                   min         mean      max         median
method                                                                     
Astrometry                       14.98    17.875000    20.77     631.180000
Eclipse Timing Variations       130.72   315.360000   500.00    4343.500000
Imaging                           7.69    67.715937   165.00   27500.000000
Microlensing                   1760.00  4144.000000  7720.00    3300.000000
Orbital Brightness Modulation  1180.00  1180.000000  1180.00       0.342887
Pulsar Timing                  1200.00  1200.000000  1200.00      66.541900
Pulsation Timing Variations        NaN          NaN      NaN    1170.000000
Radial Velocity                   1.35    51.600208   354.00     360.200000
Transit                          38.00   599.298080  8500.00       5.714932
Transit Timing Variations       339.00  1104.333333  2119.00      57.011000

**Filter**  
Uma operação de filtragem permite descartar dados com base nas propriedades do grupo.

Nota: Para que o filter funcione é necessário que se crie uma função de validação. A função ``filter()`` deve retornar um valor booleano especificando se o grupo passa pela filtragem.

In [70]:

planets.groupby('method').filter(lambda x: x['year'].mean() >= 2011)

,method,number,orbital_period,mass,distance,year
91,Transit,1,1.508956,NaN,NaN,2008
92,Transit,1,1.742994,NaN,200.00,2008
93,Transit,1,4.256800,NaN,680.00,2008
94,Transit,1,9.202050,NaN,NaN,2008
95,Transit,1,4.037896,NaN,NaN,2009
96,Transit,1,8.886593,NaN,NaN,2009
97,Transit,2,0.853585,NaN,150.00,2009
99,Transit,1,6.212290,NaN,380.00,2010
100,Transit,1,95.273800,NaN,460.00,2009
101,Transit,1,13.240600,NaN,345.00,2010
